In [22]:
from collections import defaultdict
class Graph:
    def __init__(self, connections, directed):
        self.graph = defaultdict(set)
        self.directed = directed
        self.add_connections(connections)
        
    def add_connections(self, connections):
        for node1, node2 in connections:
            self.add(node1, node2)
                
    def add(self, node1, node2):
        self.graph[node1].add(node2)
        if not self.directed:
            self.graph[node2].add(node1)
    def remove(self, node):
        for source, targets in self.graph.items():
            try:
                targets.remove(node)
            except KeyError:
                pass
        try:
            del self.graph[node]
        except KeyError:
            pass

In [23]:
import unittest
class TestGraph(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_add_connections(self):
        newConnections = [('G', 'H'), ('G', 'A')]
        self.graph.add_connections(newConnections)
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 2)
        self.assertTrue('H' in gConnections and 'A' in gConnections)
        
    def test_add(self):
        self.graph.add('G','H')
        gConnections = self.graph.graph['G']
        self.assertEqual(len(gConnections), 1)
        self.assertTrue('H' in gConnections)
        
    def test_remove(self):
        self.graph.remove('B')
        aConnections = self.graph.graph['A']
        self.assertEqual(len(aConnections), 0)
        self.assertTrue('B' not in self.graph.graph)
        
testSuiteGraph = unittest.TestLoader().loadTestsFromTestCase(TestGraph)
runner = unittest.TextTestRunner()
runner.run(testSuiteGraph)

...
----------------------------------------------------------------------
Ran 3 tests in 0.006s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

In [ ]:
def breadthFirstSearch(graph, start):
    nodesAtLevel = [start]
    visited = []
    while len(nodesAtLevel) > 0:
        nodesAtNextLevel = []
        yield nodesAtLevel
        for node in nodesAtLevel:
            targets = graph.graph[node]
            for target in targets:
                if target not in visited:
                    nodesAtNextLevel.append(target)
            visited.append(node)
        nodesAtLevel = nodesAtNextLevel
        
        
def depthFirstSearch(graph, start):
    nodesToVisit = [start]
    visited = []
    while len(nodesToVisit) > 0:
        node = nodesToVisit.pop()
        yield node
        targets = graph.graph[node]
        for target in targets:
            if target not in visited:
                nodesToVisit.append(target)
        visited.append(node)

In [26]:
import unittest
class TestSearch(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_breadthFirstSearch(self):
        index = 0
        nodesAtLevels = [['A'],['B'],['C'],['D','F'],['E','G']]
        for nodes in breadthFirstSearch(self.graph, 'A'):
            self.assertEqual(nodes, nodesAtLevels[index])
            index += 1
            
    def test_depthFirstSearch(self):
        visited = set()
        for node in depthFirstSearch(self.graph, 'A'):
            visited.add(node)
        self.assertEqual(len(visited), 7)
        
testSuiteSearch = unittest.TestLoader().loadTestsFromTestCase(TestSearch)
runner = unittest.TextTestRunner()
runner.run(testSuiteSearch)

..
----------------------------------------------------------------------
Ran 2 tests in 0.005s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

In [28]:
# 4.1 Route Between Nodes: Given a directed graph, design an algorithm to find out whether there is a route between two nodes.

# breadth first or depth first search starting from one node and looking for the other
# time complexity: O(V+E)
# space complexity: O(V)
        
def routeBetweenNodes(graph, start, end):
    for nodes in breadthFirstSearch(graph, start):
        if end in nodes:
            return True
    return False

In [27]:
import unittest
class TestRouteBetweenNodes(unittest.TestCase):
    def setUp(self):
        connections = [('A', 'B'), ('B', 'C'), ('C', 'D'), ('D', 'E'), ('C','F'), ('F', 'G')]
        self.graph = Graph(connections, directed=True)
    
    def test_routeBetweenNodes(self):
        self.assertTrue(routeBetweenNodes(self.graph, 'A', 'G'))
        self.assertFalse(routeBetweenNodes(self.graph, 'D', 'B'))
        
testSuiteRouteBetweenNodes = unittest.TestLoader().loadTestsFromTestCase(TestRouteBetweenNodes)
runner = unittest.TextTestRunner()
runner.run(testSuiteRouteBetweenNodes)

.
----------------------------------------------------------------------
Ran 1 test in 0.003s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [29]:
# source: https://stackoverflow.com/questions/1732438/how-do-i-run-all-python-unit-tests-in-a-directory
testmodules = [
    TestGraph,
    TestSearch,
    TestRouteBetweenNodes
    ]

suite = unittest.TestSuite()

for t in testmodules:
    suite.addTest(unittest.TestLoader().loadTestsFromTestCase(t))

unittest.TextTestRunner().run(suite)

......
----------------------------------------------------------------------
Ran 6 tests in 0.013s

OK


<unittest.runner.TextTestResult run=6 errors=0 failures=0>